In [2]:
# default_exp multi_condition_analysis

We build up multi-condition analyses with AlphaQuant from pairwise comparisons. We want to create the following types of analyses or overviews:

Representations where we see all samples:
- Overview of protein pseudointensities
- Z-scored type of analysis. Meaning that you see the regulation between individual samples for every protein
Wo estimate both these maps from the directLFQ algorithm

Representations where we see the individual conditions:
- protein pseudointensities per condition
- step-wise indication of up- and downregulation
- We want to show some equivalent of one-way ANOVA, which tests the null hypothesis that two or more groups have the same population mean

Representations somewhat in-between:
- pseudo intensities, such that t-test p-values have same significance as alphaquant p-values. Intensities retain the fold changes


In [3]:
#export

# class StepWiseRegulationOverview():
#     def __init__(self, results_dir, sorted_list_of_conditions):
#         self._multicond_data = #spongebob
        
